In [15]:
import duckdb
import os

conn = duckdb.connect(':memory:')

frequency = 'hourly'  # accepted values: 'daily', 'hourly'
parquet_path = f'../data/open_meteo/{frequency}/**/*.parquet'


# Schema definitions - automatically selected based on frequency
SCHEMAS = {
    'daily': {
        'date_col': 'date',
        'temp_col': 'temperature_2m_max',
        'temp_col_min': 'temperature_2m_min',
        'precip_col': 'precipitation_sum',
        'sample_cols': 'date, temperature_2m_max, temperature_2m_min, precipitation_sum',
        'description': 'Daily forecast (min/max temperatures)'
    },
    'hourly': {
        'date_col': 'timestamp',
        'temp_col': 'temperature_2m',
        'temp_col_min': None, 
        'precip_col': 'precipitation',
        'sample_cols': 'timestamp, temperature_2m, precipitation, weather_code',
        'description': 'Hourly forecast (single temperature reading)'
    }
}

# Select schema based on frequency
schema = SCHEMAS[frequency]
date_col = schema['date_col']
temp_col = schema['temp_col']
temp_col_min = schema['temp_col_min']
precip_col = schema['precip_col']
sample_cols = schema['sample_cols']

##### Schema Inspection

In [16]:
result = conn.execute(f"""SELECT * FROM parquet_scan('{parquet_path}') LIMIT 0""").pl()

result.schema

Schema([('location_name', String),
        ('requested_latitude', Float64),
        ('requested_longitude', Float64),
        ('api_latitude', Float64),
        ('api_longitude', Float64),
        ('timestamp', Datetime(time_unit='us', time_zone=None)),
        ('temperature_2m', Float64),
        ('precipitation', Float64),
        ('relative_humidity_2m', Int32),
        ('windspeed_10m', Float64),
        ('wind_direction_10m', Int32),
        ('cloud_cover', Int32),
        ('weather_code', Int32),
        ('source_api', String),
        ('ingestion_timestamp_utc', Datetime(time_unit='us', time_zone=None)),
        ('run_id', String)])

#### Completeness Check 

Verify that all 6 locations have complete data (7 records each = 7 days forecast for 'daily', likely more for 'hourly' since I let had a job running overnight at 3 hour intervals using ./examples/schedule_job.py)

In [19]:
# Count records per location
result = conn.execute(f"""
    SELECT 
        location_name,
        COUNT(*) as record_count,
        COUNT(DISTINCT {date_col}) as unique_dates
    FROM parquet_scan('{parquet_path}')
    GROUP BY location_name
    ORDER BY location_name
""").pl()

print("Records per location:")
print(result)

Records per location:
shape: (6, 3)
┌────────────────┬──────────────┬──────────────┐
│ location_name  ┆ record_count ┆ unique_dates │
│ ---            ┆ ---          ┆ ---          │
│ str            ┆ i64          ┆ i64          │
╞════════════════╪══════════════╪══════════════╡
│ Cairo          ┆ 336          ┆ 168          │
│ London         ┆ 336          ┆ 168          │
│ New York       ┆ 336          ┆ 168          │
│ Rio de Janeiro ┆ 336          ┆ 168          │
│ Sydney         ┆ 336          ┆ 168          │
│ Tokyo          ┆ 336          ┆ 168          │
└────────────────┴──────────────┴──────────────┘


##### Data Quality Checks - Temperature and Precipitation Ranges

In [21]:
# Analyze temperature and precipitation ranges by location
if temp_col_min:
    # Daily data with min/max temps
    query = f"""
    SELECT 
        location_name,
        ROUND(MIN({temp_col}), 1) as min_temp,
        ROUND(MAX({temp_col}), 1) as max_temp,
        ROUND(AVG({temp_col}), 1) as avg_temp,
        ROUND(MIN({temp_col_min}), 1) as min_temp_min,
        ROUND(MAX({precip_col}), 2) as max_precip_mm,
        COUNT(CASE WHEN {temp_col} IS NULL THEN 1 END) as null_count
    FROM parquet_scan('{parquet_path}')
    GROUP BY location_name
    ORDER BY avg_temp DESC
    """
else:
    # Hourly data with single temp
    query = f"""
    SELECT 
        location_name,
        ROUND(MIN({temp_col}), 1) as min_temp,
        ROUND(MAX({temp_col}), 1) as max_temp,
        ROUND(AVG({temp_col}), 1) as avg_temp,
        ROUND(MAX({precip_col}), 2) as max_precip_mm,
        COUNT(CASE WHEN {temp_col} IS NULL THEN 1 END) as null_count
    FROM parquet_scan('{parquet_path}')
    GROUP BY location_name
    ORDER BY avg_temp DESC
    """

result = conn.execute(query).pl()

print("Temperature and precipitation ranges:")
print(result)

# null check
null_total = result.select('null_count').sum().to_dicts()[0]['null_count']
assert null_total == 0

Temperature and precipitation ranges:
shape: (6, 6)
┌────────────────┬──────────┬──────────┬──────────┬───────────────┬────────────┐
│ location_name  ┆ min_temp ┆ max_temp ┆ avg_temp ┆ max_precip_mm ┆ null_count │
│ ---            ┆ ---      ┆ ---      ┆ ---      ┆ ---           ┆ ---        │
│ str            ┆ f64      ┆ f64      ┆ f64      ┆ f64           ┆ i64        │
╞════════════════╪══════════╪══════════╪══════════╪═══════════════╪════════════╡
│ Rio de Janeiro ┆ 23.9     ┆ 39.2     ┆ 29.9     ┆ 1.7           ┆ 0          │
│ Sydney         ┆ 11.9     ┆ 30.2     ┆ 18.2     ┆ 0.0           ┆ 0          │
│ Cairo          ┆ 7.4      ┆ 21.5     ┆ 14.7     ┆ 0.0           ┆ 0          │
│ Tokyo          ┆ -0.3     ┆ 14.6     ┆ 5.8      ┆ 0.0           ┆ 0          │
│ London         ┆ -0.3     ┆ 8.5      ┆ 4.1      ┆ 0.4           ┆ 0          │
│ New York       ┆ -7.0     ┆ 6.3      ┆ -1.7     ┆ 4.7           ┆ 0          │
└────────────────┴──────────┴──────────┴──────────┴──────

#### Ingestion Timestamps

In [ ]:
# Check ingestion timestamps - count of unique ingestion times probably more useful for daily data than hourly since hourly data may be ingested more frequently
# still nice to have since forecasts may be updated multiple times per day so allows for auditing
result = conn.execute(f"""
    SELECT 
        MIN(ingestion_timestamp_utc) as earliest_ingest,
        MAX(ingestion_timestamp_utc) as latest_ingest,
        COUNT(DISTINCT ingestion_timestamp_utc) as unique_ingest_times
    FROM parquet_scan('{parquet_path}')
""").pl()

print("Ingestion timestamps:")
print(result)

Ingestion timestamps:
shape: (1, 3)
┌────────────────────────────┬────────────────────────────┬─────────────────────┐
│ earliest_ingest            ┆ latest_ingest              ┆ unique_ingest_times │
│ ---                        ┆ ---                        ┆ ---                 │
│ datetime[μs]               ┆ datetime[μs]               ┆ i64                 │
╞════════════════════════════╪════════════════════════════╪═════════════════════╡
│ 2025-12-26 00:08:54.227224 ┆ 2025-12-26 02:39:22.568794 ┆ 12                  │
└────────────────────────────┴────────────────────────────┴─────────────────────┘


#### Compare temperature patterns across locations to verify we have realistic variation.

In [26]:
# Compare temperature and precipitation by location
if temp_col_min:
    # Daily data with min/max
    query = f"""
    SELECT 
        location_name,
        ROUND(AVG({temp_col}), 1) as avg_max_temp_c,
        ROUND(AVG({temp_col_min}), 1) as avg_min_temp_c,
        ROUND(SUM({precip_col}), 1) as total_precip_mm
    FROM parquet_scan('{parquet_path}')
    GROUP BY location_name
    ORDER BY avg_max_temp_c DESC
    """
else:
    # Hourly data with single temp
    query = f"""
    SELECT 
        location_name,
        ROUND(AVG({temp_col}), 1) as avg_temp_c,
        ROUND(MAX({precip_col}), 2) as max_precip_mm,
        COUNT(*) as record_count
    FROM parquet_scan('{parquet_path}')
    GROUP BY location_name
    ORDER BY avg_temp_c DESC
    """

result = conn.execute(query).pl()

print("Temperature and precipitation by location:")
print(result)
print("\n✓ Temperature variation shows realistic geographic differences")

Temperature and precipitation by location:
shape: (6, 4)
┌────────────────┬────────────┬───────────────┬──────────────┐
│ location_name  ┆ avg_temp_c ┆ max_precip_mm ┆ record_count │
│ ---            ┆ ---        ┆ ---           ┆ ---          │
│ str            ┆ f64        ┆ f64           ┆ i64          │
╞════════════════╪════════════╪═══════════════╪══════════════╡
│ Rio de Janeiro ┆ 29.9       ┆ 1.7           ┆ 336          │
│ Sydney         ┆ 18.2       ┆ 0.0           ┆ 336          │
│ Cairo          ┆ 14.7       ┆ 0.0           ┆ 336          │
│ Tokyo          ┆ 5.8        ┆ 0.0           ┆ 336          │
│ London         ┆ 4.1        ┆ 0.4           ┆ 336          │
│ New York       ┆ -1.7       ┆ 4.7           ┆ 336          │
└────────────────┴────────────┴───────────────┴──────────────┘

✓ Temperature variation shows realistic geographic differences


#### Samples show actual data to verify format and content.

In [ ]:
result = conn.execute(f"""
    SELECT 
        location_name,
        {sample_cols},
        ingestion_timestamp_utc
    FROM parquet_scan('{parquet_path}')
    ORDER BY location_name, {date_col}
    LIMIT 10
""").pl()

print("Sample records:")
print(result)

Sample records:
shape: (10, 6)
┌───────────────┬────────────────┬────────────────┬────────────────┬───────────────┬───────────────┐
│ location_name ┆ date           ┆ temperature_2m ┆ temperature_2m ┆ precipitation ┆ ingestion_tim │
│ ---           ┆ ---            ┆ _max           ┆ _min           ┆ _sum          ┆ estamp_utc    │
│ str           ┆ datetime[μs]   ┆ ---            ┆ ---            ┆ ---           ┆ ---           │
│               ┆                ┆ f64            ┆ f64            ┆ f64           ┆ datetime[μs]  │
╞═══════════════╪════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╡
│ Cairo         ┆ 2025-12-25     ┆ 22.8           ┆ 10.8           ┆ 0.0           ┆ 2025-12-25    │
│               ┆ 00:00:00       ┆                ┆                ┆               ┆ 23:50:22.3316 │
│               ┆                ┆                ┆                ┆               ┆ 33            │
│ Cairo         ┆ 2025-12-26     ┆ 21.3           ┆ 12.9    

#### API Metadata comaprisons vs requested (latitude, longitude, elevation, timezone) was captured correctly.

In [29]:
result = conn.execute(f"""
    SELECT DISTINCT
        location_name,
        requested_latitude,
        requested_longitude,
        api_latitude,
        api_longitude
    FROM parquet_scan('{parquet_path}')
    ORDER BY location_name
""").pl()

print(result)

shape: (6, 5)
┌────────────────┬────────────────────┬─────────────────────┬──────────────┬───────────────┐
│ location_name  ┆ requested_latitude ┆ requested_longitude ┆ api_latitude ┆ api_longitude │
│ ---            ┆ ---                ┆ ---                 ┆ ---          ┆ ---           │
│ str            ┆ f64                ┆ f64                 ┆ f64          ┆ f64           │
╞════════════════╪════════════════════╪═════════════════════╪══════════════╪═══════════════╡
│ Cairo          ┆ 30.0444            ┆ 31.2357             ┆ 30.0625      ┆ 31.25         │
│ London         ┆ 51.5074            ┆ -0.1278             ┆ 51.5         ┆ -0.12         │
│ New York       ┆ 40.7128            ┆ -74.006             ┆ 40.710335    ┆ -73.99309     │
│ Rio de Janeiro ┆ -22.9068           ┆ -43.1729            ┆ -22.875      ┆ -43.25        │
│ Sydney         ┆ -33.8688           ┆ 151.2093            ┆ -33.875      ┆ 151.125       │
│ Tokyo          ┆ 35.6895            ┆ 139.6917        